# JetRacer Version

In [2]:
#from jupyter_clickable_image_widget import ClickableImageWidget

import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

import cv2, os, glob

In [3]:
try:
    os.chdir('apex_test') # dataset's name goes here
except:
    pass
file_list = glob.glob('*.jpg')
i = 0
cur_img = file_list[i]

### Widgets

In [4]:
#Change curImgWidget to ClickableImageWidget
curImgWidget = widgets.Image(width=224, length=224)
#curImgWidget = ClickableImageWidget(width=224, length=224)
curImgWidget.value = cv2.imencode('.jpg', cv2.imread(cur_img, cv2.IMREAD_UNCHANGED))[1].tobytes()
originalXYWidget = widgets.Image(width=224, length=224)
snapshotWidget = widgets.Image(width=224, length=224)
PreButt = widgets.Button(description='Previous Image')
NextButt = widgets.Button(description='Next Image')

count_widget = ipywidgets.IntText(description='Image Position')

### Buttons

In [5]:
butt_layout = widgets.Layout(width='100px', height='80px', align_self='center')
XRightButt = widgets.Button(description='X up', layout=butt_layout)
XLeftButt = widgets.Button(description='X down', layout=butt_layout)
YUpButt = widgets.Button(description='Y up', layout=butt_layout)
YDownButt = widgets.Button(description='Y down', layout=butt_layout)
blank_pad_butt = widgets.Button(layout=butt_layout)

middle_box = widgets.HBox([XLeftButt, blank_pad_butt, XRightButt], layout=widgets.Layout(align_self='center'))
posBox = widgets.VBox([YUpButt, middle_box, YDownButt])

In [6]:
def annotate_img_cv2():
    global cur_img
    x = int(get_current_file_xy(cur_img)[0])
    y = int(get_current_file_xy(cur_img)[1])
    return cv2.imencode('.jpg', cv2.circle(cv2.imread(cur_img, cv2.IMREAD_UNCHANGED), (x, y), 8, (255, 0, 0), 3))[1].tobytes()

In [7]:
def get_current_file_xy(file_name):
    num_1 , num_2 , number_ = 0 , 1 , 0
    x, y = "", ""
    
    for i in file_name:
        if i == "_":
            number_ += 1
            if number_ == 2:
                break
        if i != "_":
            if number_ == 0:
                num_1 += 1
            num_2 += 1

    for i in range(num_1):
        x += file_name[i]
    for i in range(num_1 + 1 , num_2):
        y += file_name[i]
    xy_list = [x, y]
    return xy_list # return type is an array with two strings variables

def edit_name(img_name, x, y):
    xyList = get_current_file_xy(img_name)
    curX, curY = xyList[0], xyList[1]
    nameList = []
    for i in img_name:
        nameList.append(i)
    for i in range(len(str(curX))):
        nameList.pop(0)
    for i in range(len(str(curY))):
        nameList.pop(1)

    for i in range(len(str(y))):
        nameList.insert(i+1, str(y)[i])
    for i in range(len(str(x))):
        nameList.insert(i, str(x)[i])

    name = "".join(nameList)
    
    return name

In [8]:
def clicked(_, content, ignore):
    global cur_img, file_list
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        global i
        snapshotWidget.value = cv2.imencode('.jpg', cv2.circle(cv2.imread(file_list[i], cv2.IMREAD_UNCHANGED), (x, y), 8, (0, 255, 0), 3))[1].tobytes()
        new_name = edit_name(cur_img, x, y)
        os.rename(cur_img, new_name)
        cur_img = new_name
        file_list[i] = new_name
        
curImgWidget.on_msg(clicked)

In [9]:
#i = 0
def PreImg(change):
    global i
    global cur_img, file_list
    if (i != 0):
        i -= 1
        cur_img = file_list[i]
        curImgWidget.value = cv2.imencode('.jpg', cv2.imread(cur_img, cv2.IMREAD_UNCHANGED))[1].tobytes()
        originalXYWidget.value = annotate_img_cv2()
        count_widget.value = i

def NextImg(change):
    global i
    global cur_img, file_list
    if (i != len(file_list) - 1):
        i += 1
        cur_img = file_list[i]
        curImgWidget.value = cv2.imencode('.jpg', cv2.imread(cur_img, cv2.IMREAD_UNCHANGED))[1].tobytes()
        originalXYWidget.value = annotate_img_cv2()
        count_widget.value = i

PreButt.on_click(PreImg)
NextButt.on_click(NextImg)

In [10]:
def XLeft(ignore):
    global cur_img, file_list, i
    xyList = get_current_file_xy(cur_img)
    if int(xyList[0]) > 0:
        x, y = int(xyList[0]) - 1, int(xyList[1])
        snapshotWidget.value = cv2.imencode('.jpg', cv2.circle(cv2.imread(file_list[i], cv2.IMREAD_UNCHANGED), (x, y), 8, (0, 255, 0), 3))[1].tobytes()
        new_name = edit_name(cur_img, x, y)
        os.rename(cur_img, new_name)
        cur_img = new_name
        file_list[i] = new_name

def XRight(ignore):
    global cur_img, file_list, i
    xyList = get_current_file_xy(cur_img)
    if int(xyList[0]) < 224:
        x, y = int(xyList[0]) + 1, int(xyList[1])
        snapshotWidget.value = cv2.imencode('.jpg', cv2.circle(cv2.imread(file_list[i], cv2.IMREAD_UNCHANGED), (x, y), 8, (0, 255, 0), 3))[1].tobytes()
        new_name = edit_name(cur_img, x, y)
        os.rename(cur_img, new_name)
        cur_img = new_name
        file_list[i] = new_name

def YUp(ignore):
    global cur_img, file_list, i
    xyList = get_current_file_xy(cur_img)
    if int(xyList[1]) > 0:
        x, y = int(xyList[0]), int(xyList[1]) - 1
        snapshotWidget.value = cv2.imencode('.jpg', cv2.circle(cv2.imread(file_list[i], cv2.IMREAD_UNCHANGED), (x, y), 8, (0, 255, 0), 3))[1].tobytes()
        new_name = edit_name(cur_img, x, y)
        os.rename(cur_img, new_name)
        cur_img = new_name
        file_list[i] = new_name

def YDown(ignore):
    global cur_img, file_list, i
    xyList = get_current_file_xy(cur_img)
    if int(xyList[1]) < 224:
        x, y = int(xyList[0]), int(xyList[1]) + 1
        snapshotWidget.value = cv2.imencode('.jpg', cv2.circle(cv2.imread(file_list[i], cv2.IMREAD_UNCHANGED), (x, y), 8, (0, 255, 0), 3))[1].tobytes()
        new_name = edit_name(cur_img, x, y)
        os.rename(cur_img, new_name)
        cur_img = new_name
        file_list[i] = new_name

XLeftButt.on_click(XLeft)
XRightButt.on_click(XRight)
YUpButt.on_click(YUp)
YDownButt.on_click(YDown)

In [11]:
originalXYWidget.value = annotate_img_cv2()
display(widgets.HBox([originalXYWidget, curImgWidget, snapshotWidget, posBox]), widgets.HBox([PreButt, NextButt]), count_widget)

IntText(value=0, description='Image Position')